In [1]:
import json
import pandas as pd
import numpy as np
import time
import requests
from collections import defaultdict

In [2]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [3]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [4]:
def getLastBlockOfCurrentEpoch():
    method = 'hmy_getStakingNetworkInfo'
    params = []
    return get_information(method, params)['result']['epoch-last-block']

In [5]:
def getEpoch():
    method = 'hmy_getEpoch'
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [6]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [0]
    return get_information(method, params)['result'] 

In [7]:
def getStakeAndUndelegate(epoch):
    validator = dict()
    undelegate = dict()
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        address = i['validator']['address']
        stake = dict()
        undel = dict()
        for d in i['validator']['delegations']:
            del_address = d['delegator-address']
            del_amount = d['amount']
            if not d['undelegations']:
                undel_amount = 0
            flag = False
            for j in d['undelegations']:
                if epoch == j['epoch']:
                    flag = True
                    undel_amount = j['amount']
                    break
            if not flag:
                undel_amount = 0
            undel_num = d['undelegations']
            stake[del_address] = del_amount
            undel[del_address] = undel_amount
        validator[address] = stake
        undelegate[address] = undel
    return validator, undelegate

In [8]:
def diffAndFilter(map1, map2):
    map3 = dict()
    for key, val in map2.items():
        diff = dict()
        for k, v in map2[key].items():
            diff[k] = v - map1[key][k]
        map3[key] = diff
    return map3

In [9]:
def U1_test():
    print("Test-U1: Delegator/validator stake locked until undelegate ")
    num = int(input("enter the number of blocks you want to test: "))
    
    block = getBlockNumber()
    last_block = getLastBlockOfCurrentEpoch()
    if block + num > last_block:  
        print("current and last block numbers", block, last_block)
        print("wait until new epoch starts ...")
        new_block = last_block + 1
        while block < new_block:
            block = getBlockNumber()
    iterations = 0
    while iterations < num:
        epoch = getEpoch()
        print("current epoch number", epoch, "current block number", block, "will begin testing...")
        stake, undelegate = getStakeAndUndelegate(epoch)
        next_block = block + 1
        while block < next_block:
            block = getBlockNumber()
        epoch = getEpoch()
        print("next block reached", block, "current epoch", epoch, "will compare the stakes")
        new_stake, new_undelegate = getStakeAndUndelegate(epoch)
        diff_stake = diffAndFilter(stake, new_stake)
        diff_undelegate = diffAndFilter(undelegate, new_undelegate)
        
        reduce_num = 0
        for key, val in diff_stake.items():
            for k,v in diff_stake[key].items():
                if v < 0:
                    reduce_num += 1
                    if diff_undelegate[key][k] <= 0:
                        print("Test-U1: Fail")
                        print("Delgeator stake reduces without undelegate")
                        print("undelegate changes: ", diff_undelegate[key][k])
                        print("stake changes: ", v)
                    else:
                        print("Test-U1: Success")
        if reduce_num == 0:
            print("No stake reduces at current test, need more tests")
            
        iterations += 1        
    

In [10]:
U1_test()

Test-U1: Delegator/validator stake locked until undelegate 
enter the number of blocks you want to test: 3
current epoch number 708 current block number 26937 will begin testing...
next block reached 26938 current epoch 708 will compare the stakes
No stake reduces at current test, need more tests
current epoch number 708 current block number 26938 will begin testing...
next block reached 26939 current epoch 708 will compare the stakes
Test-U1: Success
current epoch number 708 current block number 26939 will begin testing...
next block reached 26940 current epoch 708 will compare the stakes
No stake reduces at current test, need more tests
